1° passo: construir a base de dados

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

files_path = [os.path.abspath(x) for x in os.listdir('./') if x.endswith('.png')]

def extrair_caracteristica(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    _, imgBinaria = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY) #seleciona apenas os pixels dentro do intervalo [250,255]
    momentos = cv2.moments(imgBinaria)
    momentosDeHu = cv2.HuMoments(momentos)
    feature = (-np.sign(momentosDeHu) * np.log10(np.abs(momentosDeHu)))
    return feature


base_teste = []

#extraindo as características das imagens na base de dados
for i in files_path:
    diretorio, arquivo = os.path.split(i)
    print(arquivo)
    imagem = cv2.imread(arquivo)
    carac = extrair_caracteristica(imagem)
    classe = arquivo.split('-')
    base_teste.append((carac, classe[0]))  

print('base de dados construida')    

1-.png
1-1.png
1-2.png
2-.png
2-1.png
2-2.png
3-.png
3-1.png
3-2.png
base de dados construida


2° passo: criar função chamada 'classifica' que recebe o vetor_consulta e retorna a classe que ele pertence usando K-NN

In [2]:
from statistics import mode

def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma) 

def classifica(vetor_consulta):
    d = []

    for feat in base_teste:
        vetor = feat[0]
        dist = distancia(vetor, vetor_consulta)
        d.append((dist, feat[1]))   
        
    e = sorted(d)
    k1 = e[0][1]
    k2 = e[1][1]
    k3 = e[2][1]
    a = mode([k1,k2,k3,9])
    return a

3° passo: extrair o objeto do video

In [4]:
def extrair_caracteristica_img_consulta(imgBinaria):
    momentos = cv2.moments(imgBinaria)
    momentosDeHu = cv2.HuMoments(momentos)
    feature = (-np.sign(momentosDeHu) * np.log10(np.abs(momentosDeHu)))
    return feature

In [5]:
import cv2
import matplotlib.pyplot as plt

cam = cv2.VideoCapture(0)

texto = "nao identificado"

while (True):
    ret, frame = cam.read()
    imgOriginal = frame.copy()
    
    imgBinaria = cv2.inRange(frame, (40, 40, 120), (140, 140, 255))
    #imgBinaria = cv2.erode(imgBinaria, elementoEstruturante, iterations = 2 ) 
    cv2.imshow('mask', imgBinaria)
    (lx, contornos, tree) = cv2.findContours(imgBinaria, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cp in contornos:
        if(len(cp) > 500):
            x, y, w, h = cv2.boundingRect(cp)
            # Desenhando o retângulo na imagem
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 5)
            cv2.putText(frame, texto, (100,100), cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(0, 255, 255), thickness = 2)
            vetor_consulta = extrair_caracteristica_img_consulta(imgBinaria[y:y+h, x:x+w])
            a = classifica(vetor_consulta)
            texto = "Classe: "+ a
            # Desenhando o contorno
            #cv2.drawContours(frame, cp, -1, (0, 0, 255), 2)
            
    cv2.imshow('teste', frame)
    
    
    k = cv2.waitKey(1) & 0xFF
    
    

    if k == 27: # se o usuário apertou a tecla ESC
         break
    if k == 13: # se o usuário apertou a tecla ENTER
        plt.imshow(imgOriginal[y:y+h, x:x+w, ::-1])
        plt.show()
        
        #plt.imshow(imgOriginal[y:y+h, x:x+w, ::-1])
             
    
cv2.destroyAllWindows()
cam.release()